In [2]:
'''
针对单个帧的mask的改善的处理流程。
'''
import os,glob
from wbtool import file_misc as fm
from wbtool.show import wshow,wplot
import numpy as np
from importlib import reload

cell_mask_dir = '/home/wb/samba_dir/cells_brightfield/process/lacss_point_gfp_d_e60000/' 
nuclear_mask_dir = '/home/wb/samba_dir/cells_brightfield/process/e6000_segs_cyto/' 
output_dir = '/home/wb/samba_dir/cells_brightfield/process/lacss_point_gfp_d_e60000_processed/' 

import skimage.morphology as skmorph
# from scipy import ndimage
# from skimage import measure
from onevision import morph_data, morphology
import aux
import proc_data


/home/wb/apps/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# bad cases.
# xa = fm.imread(os.path.join(data_dir, 'D1ROI1_02_3_1_GFP_001_ne4c.png'))
# print(xa.shape)
# print(xa.min(), xa.max())
# print(xa.dtype)

x1 = xa==23 #hole
x2 = xa==28 #side
x3 = xa==34 #split
x4 = xa==53 #half
x5 = xa==74 #rough
x6 = xa==80 #half

# wshow(xa==80) 

import skimage.morphology as skmorph

a = skmorph.convex_hull_image(x1)

In [ ]:
# batch process
# reload(aux)

cell_mask_dir = '/home/wb/samba_dir/cells_brightfield/process/gfp_ab_no_bv2_ne4c_e30000_dapi_segs_cyto3'

flist = glob.glob(os.path.join(cell_mask_dir, '*.png'))
# flist = ['/home/wb/samba_dir/cells_brightfield/process/lacss_point_gfp_d_e60000/D4ROI13_02_3_1_GFP_001.png']
output_dir = '/home/wb/samba_dir/cells_brightfield/process/temp/' 
save_npy = True
is_do_nuc = False

print('get {} files'.format(len(flist)))
for file in flist:
    fn = os.path.basename(file)
    print('doing', fn)

    xa = fm.imread(file)
        
    try: 
        C = morph_data.IMbind(xa, 'map')
        C = aux.fix_cell_mask(C, True, is_do_nuc, fn, nuclear_mask_dir)
    except:
        print('error encountered')
        continue

    #=== save
    # fn_out = fn+'_processed_GFP'
    # if is_do_nuc and mark == 'ne4c':
    #     fn_out += 'ne4c'
    fn_out = fn.rsplit('.', 1)[0]

    if save_npy:
        _, temp = morph_data.imbind_to_masks(C)
        np.save(os.path.join(output_dir, fn_out), temp)
    proc_data.save_prediction(morph_data.imbind_to_map(C), output_dir, fn_out+'.png')

    # d = morph_data.imbind_to_map(C)
    # proc_data.save_prediction(d, output_dir, fn)
    print('done')

